In [4]:
pip install scikit-surprise


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

In [6]:
ratings = pd.read_csv('/Users/shradhachoudhary/Downloads/Data Science 2/ratings.csv')
movies = pd.read_csv('/Users/shradhachoudhary/Downloads/Data Science 2/movies.csv')

In [7]:
ratings.head()


,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [8]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
data = pd.merge(ratings,movies, on='movieId')

In [10]:
data

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama
...,...,...,...,...,...,...
105334,668,140098,2.5,1450415424,Runoff (2015),Drama
105335,668,140816,2.5,1443288791,Tangerine (2015),Comedy|Drama
105336,668,141472,2.5,1442679119,The 50 Year Argument (2014),(no genres listed)
105337,668,142488,4.0,1451535844,Spotlight (2015),Thriller


In [11]:
reader = Reader(rating_scale=(0.5,5))

In [12]:
data = Dataset.load_from_df(data[['userId', 'movieId','rating']], reader)

In [13]:
data

In [14]:
params = {'name':'cosine', 'user_based':False}

In [15]:
model = KNNBasic(sim_options=params)

In [16]:
cross_validate(model, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9784  0.9691  0.9736  0.9773  0.9701  0.9737  0.0037  
MAE (testset)     0.7625  0.7531  0.7571  0.7594  0.7526  0.7569  0.0038  
Fit time          5.65    2.84    2.85    4.14    4.39    3.97    1.06    
Test time         2.47    2.40    2.66    2.89    2.30    2.54    0.21    


{'test_rmse': array([0.97842274, 0.96907417, 0.97359306, 0.97730727, 0.97013702]),
 'test_mae': array([0.76245241, 0.75305915, 0.75712742, 0.75938343, 0.75257698]),
 'fit_time': (5.650238990783691,
  2.83561110496521,
  2.8455018997192383,
  4.1401238441467285,
  4.393479108810425),
 'test_time': (2.4656431674957275,
  2.4036242961883545,
  2.6597681045532227,
  2.8880529403686523,
  2.3011178970336914)}

In [17]:
from surprise import accuracy
from surprise.model_selection import train_test_split

In [18]:
trainset, testset = train_test_split(data,test_size=0.2)

In [19]:
model2 = KNNBasic(sim_options={'user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9504


0.9503634784234971

In [20]:
model2 = KNNBasic(min_k = 5, k=10, sim_options={'name':'cosine','user_based':True})
model2.fit(trainset)
predictions = model2.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9852


0.9852293727168304

In [21]:
model2.predict(668,16)

Prediction(uid=668, iid=16, r_ui=None, est=4.0493808465450085, details={'actual_k': 10, 'was_impossible': False})

In [22]:
movies[movies['movieId']==16]

,movieId,title,genres
15,16,Casino (1995),Crime|Drama


In [23]:
final_dataset = pd.merge(movies,ratings, on='movieId')

In [24]:
final_dataset['predicted_rating'] = final_dataset.apply(lambda x:model2.predict(x['userId'],x['movieId']).est, axis=1)

In [25]:
final_dataset.head()

,movieId,title,genres,userId,rating,timestamp,predicted_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,3.900000
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,3.750000
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,3.450000
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.950033
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.100000


In [26]:
# SVD algo

from surprise import SVD

In [27]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8711  0.8673  0.8636  0.8711  0.8695  0.8685  0.0028  
Fit time          0.41    0.40    0.40    0.41    0.40    0.40    0.00    
Test time         0.04    0.09    0.04    0.04    0.04    0.05    0.02    


{'test_rmse': array([0.87107296, 0.86725494, 0.86364127, 0.8710595 , 0.86946682]),
 'fit_time': (0.4055359363555908,
  0.4010591506958008,
  0.40227580070495605,
  0.4059920310974121,
  0.40056276321411133),
 'test_time': (0.03734993934631348,
  0.08875894546508789,
  0.03637290000915527,
  0.03694510459899902,
  0.036382198333740234)}

In [28]:
trainset = data.build_full_trainset()

In [29]:
svd.fit(trainset)

In [30]:
final_dataset['svd_pred'] = final_dataset.apply(lambda x: svd.predict(x['userId'],x['movieId']).est, axis=1)

In [33]:
final_dataset

,movieId,title,genres,userId,rating,timestamp,predicted_rating,svd_pred
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895,3.900000,4.198766
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039,3.750000,3.754667
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933,3.450000,4.174457
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810,3.950033,3.693877
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286,4.100000,3.939488
...,...,...,...,...,...,...,...,...
105334,148238,A Very Murray Christmas (2015),Comedy,475,3.0,1451213043,3.516702,3.010666
105335,148626,The Big Short (2015),Drama,458,4.0,1452014749,3.516702,3.702687
105336,148626,The Big Short (2015),Drama,576,4.5,1451687664,3.516702,4.041207
105337,148626,The Big Short (2015),Drama,668,4.5,1451148148,3.516702,3.156173
